In [4]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import zeros, newaxis
import cv2

import matplotlib.pyplot as plt
import seaborn

ModuleNotFoundError: No module named 'cv2'

In [5]:
abnormal_path_train =  'MRNet-v1.0/train-abnormal.csv'
acl_path_train = 'MRNet-v1.0/train-acl.csv'
meniscus_train =  'MRNet-v1.0/train-meniscus.csv'


abnormal_path_valid =  'MRNet-v1.0/valid-abnormal.csv'
acl_path_valid = 'MRNet-v1.0/valid-acl.csv'
meniscus_valid =  'MRNet-v1.0/valid-meniscus.csv'

In [6]:
def load_labels(abnormal_path, acl_path, meniscus_path):

    df_labels_abnormal =  pd.read_csv(abnormal_path, names=['Image_Name', 'Abnormal_Label'])
    df_labels_acl = pd.read_csv(acl_path, names=['Image_Name', 'Acl_Label'])
    df_labels_meniscus = pd.read_csv(meniscus_path, names=['Image_Name', 'Meniscus_Label'])

    df_lables_combined = pd.merge(df_labels_abnormal, df_labels_acl, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined = pd.merge(df_lables_combined, df_labels_meniscus, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].astype(str)
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].apply(lambda x: x.zfill(4))

    return df_lables_combined

In [7]:
df_labels_train = load_labels(abnormal_path_train, acl_path_train, meniscus_train)
df_labels_train

df_labels_valid = load_labels(abnormal_path_valid, acl_path_valid, meniscus_valid)
df_labels_valid

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,1130,0,0,0
1,1131,0,0,0
2,1132,0,0,0
3,1133,0,0,0
4,1134,0,0,0
...,...,...,...,...
115,1245,1,1,1
116,1246,1,1,1
117,1247,1,0,1
118,1248,1,1,1


In [8]:
# Read Traning Data
def read_data(train_path, df_labels):
    axial_images = []
    coronal_images = []
    sagittal_images = []

    axial_labels = []
    coronal_labels = []
    sagittal_labels = []

    # train_path = '/kaggle/input/mrnet-v1/MRNet-v1.0/train'
    train_folders = glob.glob(train_path + '/*')

    for folder in train_folders:
        train_data = glob.glob(folder + '/*.npy')
        print('Total samples in data:', len(train_data))
        print('Loading Data From :   ', folder)

        for i in train_data:
            #             print(i)
            img_array = np.load(i)
            start, end = (img_array.shape[0]/2)-8, (img_array.shape[0]/2)+8
            image_tensor = img_array[int(start):int(end),:,:].reshape(256,256,16)
            if 'axial' in i:
                axial_images.append(image_tensor)
                image_name = i.rsplit('/')[-1].split('.')[0]
                axial_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])
            if 'coronal' in i:
                coronal_images.append(image_tensor)
                image_name = i.rsplit('/')[-1].split('.')[0]
                coronal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])
            if 'sagittal' in i:
                sagittal_images.append(image_tensor)
                image_name = i.rsplit('/')[-1].split('.')[0]
                sagittal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])

        print('Loading Complete From :   ', folder)
        print('\n\n')
    return axial_images, coronal_images, sagittal_images, axial_labels

train_path =  'MRNet-v1.0/train'
valid_path =  'MRNet-v1.0/valid'